In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from utils import get_unfindicators, fillna__
pd.options.io.excel.xlsx.writer = 'xlsxwriter'

target = r"C:\datas\企业绩效数据\非财务绩效数据\deals"
unfinancials, indicator_dict = get_unfindicators()
china_dataset = pd.DataFrame()
# china_dataset = pd.read_csv(r'C:\datas\企业信息数据\中国上市制造企业汇总表(简版).txt', index_col=0, dtype={'股票代码': "object"})
# sckds = set(china_dataset['股票代码'].to_list())
# china_dataset.rename(columns={'股票代码': 'Symbol', '省份': '上市省份'}, inplace=True)
# 数据合并
files = [os.path.join(target, entry.name) for entry in os.scandir(target)]
for i in range(len(files)):
    dataset = pd.read_csv(files[i], sep='\t', header=0, dtype={'Symbol': 'object'})
    # dataset = dataset[dataset['Symbol'].isin(sckds)]
    if 'EndDate' not in china_dataset.columns:
        china_dataset = dataset
    else:
        china_dataset = pd.merge(china_dataset, dataset, on=['Symbol', 'EndDate'], how='outer')

name_dic = {name: indicator_dict[name] for name in china_dataset.columns.values.tolist()}
china_dataset.rename(columns=name_dic, inplace=True)
china_dataset.rename(columns={'统计截止日期': '截止年份'}, inplace=True)
# china_dataset.dropna(subset=['上市省份'], inplace=True)

# 有些数据不应该是小数，因此不能通过线性插值法填充数据
# china_dataset['董事人数'].fillna(0, inplace=True)
# china_dataset['独立董事人数'].fillna(0, inplace=True)
# china_dataset['董事人数'] = china_dataset['董事人数'].astype('int32')
# china_dataset['独立董事人数'] = china_dataset['独立董事人数'].astype('int32')
china_dataset['独董比例'] = china_dataset['独立董事人数']/china_dataset['董事人数']
china_dataset['董事会规模'].fillna(0, inplace=True)
china_dataset['董事会规模'] = china_dataset['董事会规模'].astype('int32')
china_dataset['两职合一'].fillna(0, inplace=True)
china_dataset['股权性质'].fillna(0, inplace=True)
china_dataset['两职合一'] = china_dataset['两职合一'].astype('int32')
china_dataset['两职合一'] = china_dataset['两职合一'].apply(lambda x:0 if x==1 else 1)
china_dataset['股权性质'] = china_dataset['股权性质'].astype('int32')
china_dataset.drop(['董事人数', '独立董事人数'], axis=1, inplace=True)
# 注意：职工人数；授权专利数也应该是整数
# china_dataset.to_csv(r'C:\onedrivers\OneDrive - jsnu.edu.cn\projects\digital\06.因果机制研究\data\非财务绩效数据.csv',index=False,encoding='utf8')

# 删除部分缺失值太多的变量
china_dataset = china_dataset.drop(['授权专利数','授权专利数2','研发人员数量', '研发投入金额'], axis=1)
# 删除一些不需要的数据/'纳税总额A',
china_dataset = china_dataset.drop(['内部控制指数','长期绩效增长', '组织韧性', '财务波动','两职合一','股权性质'], axis=1)
dataset = fillna__(china_dataset, '股票代码',rate=0.2)  # 先对数据进行一次填充


dataset.to_csv(r'./datas/全国上市企业非财务绩效汇总表.csv',encoding='utf8',index=None)
china_dataset

,截止年份,股票代码,董事会规模,员工人数,纳税总额A,捐赠总额,本期销售额占年度销售总额比例(%),本期采购额占年度采购总额比例(%),主营业务收入,职工薪酬,两权分离度(%),股权集中度4(%),研发人员数量占比(%),研发营收比(%),独董比例
0,2007,000001,250426,8573.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.9984,NaN,NaN,0.285714
1,2008,000001,295216,10381.0,NaN,1500.0,NaN,NaN,NaN,NaN,NaN,34.7276,NaN,NaN,0.285714
2,2009,000001,230571,11308.0,NaN,1300.0,NaN,NaN,NaN,NaN,NaN,33.7005,NaN,NaN,0.333333
3,2010,000001,352655,12203.0,308400.0,580.3,NaN,NaN,NaN,NaN,NaN,39.5633,NaN,NaN,0.312500
4,2011,000001,272946,13778.0,473200.0,198.0,NaN,NaN,NaN,NaN,NaN,58.9121,NaN,NaN,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50624,2018,900957,24833,75.0,NaN,NaN,100.0,87.0,6.470736e+07,3812398.46,13.0328,48.0287,NaN,NaN,0.333333
50625,2019,900957,23928,78.0,NaN,NaN,100.0,94.0,1.050223e+08,4830802.38,13.0328,48.1793,NaN,NaN,0.333333
50626,2020,900957,23191,78.0,NaN,NaN,100.0,86.0,NaN,5517133.22,13.0328,48.3093,NaN,NaN,0.454545
50627,2021,900957,22442,78.0,NaN,NaN,100.0,84.0,NaN,5808021.88,13.0328,48.3658,NaN,NaN,0.333333


In [3]:
import numpy as np

dataset['人力资源效率'] = dataset['主营业务收入'] / dataset['员工人数']
# dataset['独董比例'] = dataset['独立董事人数'] / dataset['董事人数']
# dataset['供应链集成(%)'] = (dataset['本期销售额占年度销售总额比例(%)'] + dataset['本期采购额占年度采购总额比例(%)']) / 2
dataset['提供岗位增长率'] = dataset.groupby('股票代码')['员工人数'].pct_change(1, fill_method='ffill')
# dataset['企业税费增长率'] = dataset.groupby('股票代码')['纳税总额A'].pct_change(1, fill_method='ffill')
dataset['员工收入增长率'] = dataset.groupby('股票代码')['职工薪酬'].pct_change(1, fill_method='ffill')

dataset['提供岗位增长率'].fillna(dataset['提供岗位增长率'].mean(), inplace=True)
dataset['员工收入增长率'].fillna(dataset['员工收入增长率'].mean(), inplace=True)
dataset['提供岗位增长率'].replace(to_replace=np.inf, value=dataset['提供岗位增长率'].mean(), inplace=True)
dataset['员工收入增长率'].replace(to_replace=np.inf, value=dataset['提供岗位增长率'].mean(), inplace=True)
# dataset['研发强度'] = dataset['研发营收比(%)'].apply(lambda x:np.log(1+x))
# dataset['创新产出'] = dataset['授权专利数2'].apply(lambda x:np.log(1+x))
dataset['董事会规模'] = dataset['董事人数'].apply(lambda x: np.log(1+x))
dataset.drop(['主营业务收入', '员工人数', '董事人数','研发营收比(%)', '独立董事人数', '职工薪酬','授权专利数2'], axis=1, inplace=True)
# '本期销售额占年度销售总额比例(%)', '本期采购额占年度采购总额比例(%)',
dataset

,截止年份,股票代码,两职合一,本期销售额占年度销售总额比例(%),本期采购额占年度采购总额比例(%),股权性质,两权分离度(%),股权集中度4(%),研发人员数量占比(%),人力资源效率,独董比例,提供岗位增长率,员工收入增长率,研发强度,创新产出,董事会规模
0,2007,000001,0,33.246757,35.87862,0,4.880392,30.9984,NaN,NaN,0.285714,inf,0.161442,2.118253,0.000000,2.708050
1,2008,000001,0,33.246757,35.87862,0,4.880392,34.7276,NaN,NaN,0.285714,0.210895,0.161442,2.118253,0.000000,2.708050
2,2009,000001,0,33.246757,35.87862,0,4.880392,33.7005,NaN,NaN,0.333333,0.089298,0.161442,2.118253,0.000000,2.772589
3,2010,000001,1,33.246757,35.87862,0,4.880392,39.5633,NaN,NaN,0.312500,0.079148,0.161442,2.118253,1.609438,2.833213
4,2011,000001,1,33.246757,35.87862,0,4.880392,58.9121,NaN,NaN,0.444444,0.129067,0.161442,2.118253,0.000000,2.944439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48234,2018,900957,1,100.000000,87.00000,0,13.032800,48.0287,NaN,8.627647e+05,0.333333,-0.038462,-0.178979,2.118253,0.000000,2.302585
48235,2019,900957,1,100.000000,94.00000,0,13.032800,48.1793,NaN,1.346440e+06,0.333333,0.040000,0.267129,2.118253,0.000000,2.302585
48236,2020,900957,1,100.000000,86.00000,0,13.032800,48.3093,NaN,NaN,0.454545,0.000000,0.142074,2.118253,0.000000,2.484907
48237,2021,900957,1,100.000000,84.00000,0,13.032800,48.3658,NaN,NaN,0.333333,0.000000,0.052725,2.118253,0.000000,1.945910


In [4]:
dataset.sort_values(['股票代码', '截止年份'], inplace=True)
dataset.reset_index(drop=True, inplace=True)
# dataset.drop(['上市日期','上市省份'],inplace=True,axis=1)
dataset

,截止年份,股票代码,两职合一,本期销售额占年度销售总额比例(%),本期采购额占年度采购总额比例(%),股权性质,两权分离度(%),股权集中度4(%),研发人员数量占比(%),人力资源效率,独董比例,提供岗位增长率,员工收入增长率,研发强度,创新产出,董事会规模
0,2007,000001,0,33.246757,35.87862,0,4.880392,30.9984,NaN,NaN,0.285714,inf,0.161442,2.118253,0.000000,2.708050
1,2008,000001,0,33.246757,35.87862,0,4.880392,34.7276,NaN,NaN,0.285714,0.210895,0.161442,2.118253,0.000000,2.708050
2,2009,000001,0,33.246757,35.87862,0,4.880392,33.7005,NaN,NaN,0.333333,0.089298,0.161442,2.118253,0.000000,2.772589
3,2010,000001,1,33.246757,35.87862,0,4.880392,39.5633,NaN,NaN,0.312500,0.079148,0.161442,2.118253,1.609438,2.833213
4,2011,000001,1,33.246757,35.87862,0,4.880392,58.9121,NaN,NaN,0.444444,0.129067,0.161442,2.118253,0.000000,2.944439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50624,2018,900957,1,100.000000,87.00000,0,13.032800,48.0287,NaN,8.627647e+05,0.333333,-0.038462,-0.178979,2.118253,0.000000,2.302585
50625,2019,900957,1,100.000000,94.00000,0,13.032800,48.1793,NaN,1.346440e+06,0.333333,0.040000,0.267129,2.118253,0.000000,2.302585
50626,2020,900957,1,100.000000,86.00000,0,13.032800,48.3093,NaN,NaN,0.454545,0.000000,0.142074,2.118253,0.000000,2.484907
50627,2021,900957,1,100.000000,84.00000,0,13.032800,48.3658,NaN,NaN,0.333333,0.000000,0.052725,2.118253,0.000000,1.945910


In [5]:
dataset.to_csv(r'./datas/全国上市企业非财务绩效汇总表.csv',encoding='utf8',index=None)